In [1]:
from sklearn import preprocessing
from sklearn import metrics
import pandas as pd
from sklearn.model_selection import KFold
from IPython.display import display
from ipywidgets import IntProgress
import pandas as pd
import os
from sklearn.model_selection import RandomizedSearchCV
from time import time
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Perceptron

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [3]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))
set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,'..','Set de entrenamiento, testing y predicción.ipynb'))
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')
pd.options.mode.chained_assignment = None


In [4]:
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [5]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

In [6]:
%run $set_de_entrenamiento_testing_y_prediccion

labels_with_features = labels.merge(df_features, how='inner', on='person')
data = labels_with_features.drop('label', axis=1)
target = labels_with_features['label']

In [7]:
param_dist = {
    'penalty' : [None,'l2','l1','elasticnet'],
    'alpha' : np.arange(0.0001,0.005,0.001)

}

cv_splits = 100 # cantidad de splits en el cross validation
n_iter_search = 20 # cantidad de puntos, en total splits*n_iter_search 

perceptron = Perceptron()

random_search = RandomizedSearchCV(perceptron, param_distributions=param_dist, iid=False, refit=True, verbose=10,
                                   return_train_score=True, n_iter=n_iter_search, cv=cv_splits,
                                   scoring='roc_auc', n_jobs=-1);
start = time()
random_search.fit(data, target)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

Fitting 100 folds for each of 20 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:   

RandomizedSearchCV took 222.57 seconds for 20 candidates parameter settings.


In [8]:
random_search.cv_results_

{'mean_fit_time': array([0.26433931, 0.25896271, 0.36754014, 0.37516719, 0.25642705,
        0.26110337, 0.39752023, 0.38787769, 0.25846886, 0.25613314,
        0.44847685, 0.39165862, 0.25182173, 0.25598335, 0.57428787,
        0.39694783, 0.25359199, 0.24827761, 0.65693116, 0.38722181]),
 'std_fit_time': array([0.07029916, 0.05783222, 0.09746506, 0.10802889, 0.05444903,
        0.05937925, 0.12176972, 0.10701203, 0.05362929, 0.06029388,
        0.16052331, 0.11588375, 0.05473329, 0.0544839 , 0.25311226,
        0.11560882, 0.05731598, 0.06612246, 0.31967021, 0.1160695 ]),
 'mean_score_time': array([0.0061782 , 0.00607319, 0.00449718, 0.00433565, 0.00581882,
        0.00543496, 0.0049258 , 0.00504481, 0.00580559, 0.00540735,
        0.00433304, 0.00507596, 0.00525405, 0.00563583, 0.0047533 ,
        0.00480785, 0.00602311, 0.00530936, 0.00409411, 0.00448383]),
 'std_score_time': array([0.0034353 , 0.00297378, 0.00206797, 0.00178891, 0.00283323,
        0.00205811, 0.0024319 , 0.002539

In [9]:
random_search.best_params_

{'penalty': 'l1', 'alpha': 0.0041}

In [10]:
random_search.best_score_

0.701236936936937